In [ ]:
import os
import pandas as pd
from glob import glob
from tsfresh import extract_features

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

jobs = multiprocessing.cpu_count()
print("CPU count: "+ str(jobs))

output_file = 'tmp/extracted_data_test.csv'

for i, file in enumerate(files[:5]):
    print("Extracting file: " + str(i) + " (" + file + ")")
    df = pd.read_csv(file)
    df['id'] = i
    target = df.TOTAL_QUALITY.unique()[0]
    df = df.drop(axis=1, columns=totals)

    consta = [col for col in df if col.endswith(('K1', 'K2', 'K3'))]
    series = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]

    extracted_features = extract_features(df[series], disable_progressbar=True, column_id='id', column_sort='Time', n_jobs=jobs)

    for j in consta:
        extracted_features[j] = df[j].unique()[0]    
    
    extracted_features['target'] = target    
    
    if not os.path.isfile(output_file):
        extracted_features.to_csv(output_file)
    else:
        extracted_features.to_csv(output_file, mode='a', header=False)

print('Tsfresh succesfully finished')

CPU count: 8
Extracting file: 0 (data\0.csv)
Extracting file: 1 (data\1.csv)
Extracting file: 2 (data\10.csv)
Extracting file: 3 (data\100.csv)


In [9]:
df = pd.read_csv('tmp/extracted_data_test.csv')
df

,id,Flow rate__abs_energy,Flow rate__absolute_sum_of_changes,"Flow rate__agg_autocorrelation__f_agg_""mean""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""median""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""var""__maxlag_40","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""",...,Zone29_K1,Zone29_K2,Zone29_K3,Zone30_K1,Zone30_K2,Zone30_K3,Zone31_K1,Zone31_K2,Zone31_K3,target
0,0,1.031236e-07,0.000062,0.0,0.0,0.0,0.000012,-0.243467,-4.506888e-08,1.958974e-08,...,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,1
1,1,1.042159e-07,0.000064,0.0,0.0,0.0,0.000012,-0.253883,-4.662547e-08,1.926688e-08,...,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0


In [ ]:
import numpy as np
from glob import glob
import pandas as pd
import csv
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

df_list = list()
target = list()

for i, file in enumerate(files):
    df = pd.read_csv(file)

    if df.iloc[:,-1].unique()[0] == target_value:
        df['id'] = i
        target.append(df.TOTAL_QUALITY.unique()[0])
        df = df.drop(axis=1, columns=totals)   

        consta = [col for col in df if col.endswith(('K1', 'K2', 'K3'))]
        series = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]  

        distributor = MultiprocessingDistributor(n_workers=8, disable_progressbar=True, progressbar_title="Feature Extraction")
        extracted_features = extract_features(df[series], column_id='id', column_sort='Time', distributor=distributor)

        for j in consta:
            extracted_features[j] = df[j].unique()[0] 

        df_list.append(extracted_features)   



df = pd.concat(df_list)
df.head()        